# 🎛️ Damped Harmonic Oscillator – Interactive Exploration

This notebook illustrates the **damped harmonic oscillator** — a fundamental model derived from *Newton’s second law* that captures oscillatory systems under different damping conditions.

It allows you to interactively explore:
- the effect of mass, stiffness, and damping on the motion,  
- underdamped, critically damped, and overdamped regimes,  
- and how the oscillation envelope evolves over time.

---

👉 **Open this notebook in Google Colab:**
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lrhgit/fetalcry/blob/main/Damped_harmonic_oscillator.ipynb)


## ⚙️ Physical Motivation and Model

The model is based on **Newton’s second law**, where the sum of forces acting on a mass equals its acceleration:

$$
m \ddot{x} = -k x - c \dot{x}
$$

Here  
- $m$: mass (inertia)  
- $k$: stiffness (elastic restoring force)  
- $c$: damping coefficient (viscous or resistive losses)  
- $x(t)$: displacement as a function of time  

Rearranging gives the standard **second-order differential equation**:

$$
\ddot{x} + 2\zeta \omega_0 \dot{x} + \omega_0^2 x = 0
$$

with

$$
\omega_0 = \sqrt{\frac{k}{m}}, \qquad 
\zeta = \frac{c}{2\sqrt{km}}
$$

where $\omega_0$ is the **natural (undamped) angular frequency**,  
and $\zeta$ (zeta) is the **dimensionless damping ratio**.

In [5]:
#@title 🛠️ Setup (click to expand)
# Automatically installs ipywidgets if missing (for Colab)
try:
    import ipywidgets
except ModuleNotFoundError:
    !pip install -q ipywidgets

In [6]:
#@title 🎛 Interactive Damped Oscillator 
#@markdown then press “Run cell” to explore the damping behaviour.

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, Dropdown, Checkbox, HBox, VBox, interactive_output

# ---------- helper functions ----------
def zeta_omega0(m, k, c):
    omega0 = np.sqrt(k / m)
    zeta = c / (2 * np.sqrt(k * m))
    return zeta, omega0

def harmonic_amps(N, law="equal", normalize=True):
    n = np.arange(0, N)
    if law == "equal":
        A = np.ones_like(n, dtype=float)
    elif law == "1/n":
        A = np.ones_like(n, dtype=float)
        A[1:] = 1.0 / n[1:]
    elif law == "1/n^2":
        A = np.ones_like(n, dtype=float)
        A[1:] = 1.0 / (n[1:]**2)
    else:
        A = np.ones_like(n, dtype=float)
    if normalize and A.sum() > 0:
        A /= A.sum()
    return A

def mode_response(omega_n, zeta, A_n, t):
    if zeta < 1.0:
        omega_d = omega_n * np.sqrt(1.0 - zeta**2)
        y = A_n * np.exp(-zeta * omega_n * t) * (
            np.cos(omega_d * t)
            + (zeta / np.sqrt(1 - zeta**2)) * np.sin(omega_d * t)
        )
        env = A_n * np.exp(-zeta * omega_n * t)
        return y, env
    else:
        y = A_n * np.exp(-zeta * omega_n * t)
        return y, None

def plot_response(m, k, c, n_periods, N_harm, scale_law, do_norm):
    zeta, omega0 = zeta_omega0(m, k, c)
    omega_d = omega0 * np.sqrt(1 - zeta**2) if zeta < 1 else omega0
    period = 2 * np.pi / omega_d
    t_end = n_periods * period
    t = np.linspace(0, t_end, 2000)

    A = harmonic_amps(N_harm, law=scale_law, normalize=do_norm)
    colors = plt.cm.viridis(np.linspace(0, 1, N_harm))

    plt.figure(figsize=(9, 4.5))
    for i in range(N_harm):
        omega_n = (i + 1) * omega0
        y_n, env = mode_response(omega_n, zeta, A[i], t)
        plt.plot(t, y_n, color=colors[i], lw=1.6, label=f"n={i}")
        if (zeta < 1) and (env is not None):
            plt.plot(t, env, "--", color=colors[i], alpha=0.6)

    plt.title(f"ζ = {zeta:.3f}   ω₀ = {omega0:.2f} rad/s   (scaling: {scale_law}, normalized: {do_norm})")
    plt.xlabel("time")
    plt.ylabel("amplitude per harmonic")
    plt.grid(True, alpha=0.3)
    plt.legend(ncol=3, fontsize=8, frameon=False)
    plt.tight_layout()
    plt.show()

# ---------- widgets ----------
# Left column: physical parameters
m_slider = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="m")
k_slider = FloatSlider(value=10.0, min=1.0, max=100.0, step=1.0, description="k")
c_slider = FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description="c")

# Right column: plot controls
nperiods_slider = IntSlider(value=3, min=1, max=30, step=1, description="periods")
Nh_slider = IntSlider(value=2, min=1, max=12, step=1, description="# harmonics")
scale_dd = Dropdown(options=["equal", "1/n", "1/n^2"], value="1/n^2", description="scaling")
norm_cb = Checkbox(value=True, description="normalize ΣAₙ = 1")

# Layout: plot on top, sliders below in two columns
ui_left = VBox([m_slider, k_slider, c_slider])
ui_right = VBox([nperiods_slider, Nh_slider, scale_dd, norm_cb])
ui_sliders = HBox([ui_left, ui_right])

out = interactive_output(
    plot_response,
    dict(m=m_slider, k=k_slider, c=c_slider,
         n_periods=nperiods_slider,
         N_harm=Nh_slider,
         scale_law=scale_dd,
         do_norm=norm_cb)
)

display(VBox([out, ui_sliders]))

## 🌊 Oscillatory Regimes

The dynamic behavior of the system depends on the value of the **damping ratio**, $ \zeta $.
It determines whether the response is oscillatory or purely exponential.

| Regime | Condition | Characteristic Response | Physical meaning |
|:--|:--|:--|:--|
| **Underdamped** | $ \zeta < 1 $ | Oscillatory motion with exponentially decaying amplitude | Energy loss per cycle is small; the system vibrates freely before coming to rest |
| **Critically damped** | $ \zeta = 1 $ | Fastest return to equilibrium without oscillation | System returns to rest as quickly as possible, no overshoot |
| **Overdamped** | $ \zeta > 1 $ | Non-oscillatory, slow exponential decay | Damping dominates; motion is sluggish and aperiodic |

For the **underdamped** case, the displacement can be written as:

$$
x(t) = A_0 e^{-\zeta \omega_0 t} \cos(\omega_d t + \phi)
$$

where

$$
\omega_d = \omega_0 \sqrt{1 - \zeta^2}
$$

is the **damped angular frequency**, slightly lower than the natural frequency $ \omega_0 $.

The exponential term $ e^{-\zeta \omega_0 t} $ defines the **envelope** of the oscillation,
illustrating how energy dissipates over time.  
This rate of decay is directly linked to viscous or structural losses in real systems.

## 💡 Interpretation

Although this is a simple **one-degree-of-freedom (1-DOF)** oscillator,
it captures essential physics of more complex **Fluid–Structure Interaction (FSI)** systems.

- The **mass** $m$ represents inertia, such as tissue or structural mass resisting acceleration.  
- The **stiffness** $k$ represents the restoring elasticity that tends to bring the structure back to its equilibrium shape.  
- The **damping** $c$ captures energy losses — either due to internal tissue viscosity, surrounding fluid drag, or mechanical resistance.  

In the context of **vocal fold dynamics** or **fetal airway modeling**,  
these parameters collectively determine whether oscillations can be **self-sustained**, **damped out**,  
or **critically balanced** near the onset of phonation-like motion.

The damping ratio $ \zeta $ thus provides a compact, dimensionless measure of the balance between  
energy storage (via $k$) and energy dissipation (via $c$).  
Small $ \zeta $ values lead to sustained vibration — a prerequisite for sound generation —  
while high $ \zeta $ suppresses oscillations entirely.

This makes the model a useful conceptual foundation for understanding  
**how viscous and elastic effects influence vibration frequency, amplitude, and decay**  
in FSI problems such as vocal fold or membrane motion.